# Accessing storage in Databricks
---
## 0. Prerequisites
### 0.1. Azure App registration

1. [Create an Azure AD application and service principal that can access resources](https://docs.microsoft.com/en-us/azure/azure-resource-manager/resource-group-create-service-principal-portal). Note the following properties:
  - application-id: An ID that uniquely identifies the application
  - directory-id: An ID that uniquely identifies the Azure AD instance
  - storage-account-name: The name of the storage account
  - service-credential: A string that the application uses to prove its identity   
  
2. Register the service principal, granting the correct role assignment, such as Storage Blob Data Contributor, on the Azure Data Lake Storage Gen2 account.

### 0.2. Adding scoped secrets
To add a secret and a scope, this needs to be completed using the [Databricks CLI](https://docs.databricks.com/dev-tools/cli/index.html).

Once signed in via the CLI, issue the following commands:
1. `databricks secrets create-scope --scope Analysts`
2. `databricks secrets put --scope Analysts --key SPID --string-value "Service Principal ID"` (Application Client ID)
3. `databricks secrets put --scope Analysts --key SPKey --string-value "Service Principal Secret Key"`
4. `databricks secrets put --scope Analysts --key DirectoryID --string-value "Azure Directory ID"`

## 1. Mount Azure Data Lake Storage Gen2 filesystem
This only needs to be done once per cluster

In [3]:
# service principal details
client_id     = dbutils.secrets.get(scope='Analysts', key='SPID')  # 'cb90462d-aaae-4038-ac60-b8e090d404bb'
client_secret = dbutils.secrets.get(scope='Analysts', key='SPKey')  # '-nGdTkdSa-TN-4Fr7z0aBX3.k4Dct54SOX'
directory_id  = dbutils.secrets.get(scope='Analysts', key='DirectoryID')  # '600fc753-881c-4179-a38f-27842e71dc98'

# data lake container details
account_name = 'storagejamesleslie'
container_name = 'ecovacs'

configs = {'fs.azure.account.auth.type': 'OAuth',
           'fs.azure.account.oauth.provider.type': 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider',
           'fs.azure.account.oauth2.client.id': client_id,
           'fs.azure.account.oauth2.client.secret': client_secret,
           'fs.azure.account.oauth2.client.endpoint': f'https://login.microsoftonline.com/{directory_id}/oauth2/token'}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
    source = f'abfss://{container_name}@{account_name}.dfs.core.windows.net/',
    mount_point = f'/mnt/{container_name}',
    extra_configs = configs)

# list all items in container
display(dbutils.fs.ls(f'/mnt/{container_name}'))

Out[4]: [FileInfo(path='dbfs:/mnt/ecovacs/BestBuy/', name='BestBuy/', size=0),
 FileInfo(path='dbfs:/mnt/ecovacs/VSI/', name='VSI/', size=0)]

### Unmount a mount point

In [5]:
# unmount container
dbutils.fs.unmount(f'/mnt/{container_name}')

/mnt/ecovacs has been unmounted.
Out[3]: True

## 2. Access directly with service principal and OAuth 2.0
All code below needs to be run every time

In [7]:
# service principal details
client_id     = dbutils.secrets.get(scope='Analysts', key='SPID')  # 'cb90462d-aaae-4038-ac60-b8e090d404bb'
client_secret = dbutils.secrets.get(scope='Analysts', key='SPKey')  # '-nGdTkdSa-TN-4Fr7z0aBX3.k4Dct54SOX'
directory_id  = dbutils.secrets.get(scope='Analysts', key='DirectoryID')  # '600fc753-881c-4179-a38f-27842e71dc98'

# update spark configs
spark.conf.set('fs.azure.account.auth.type', 'OAuth')
spark.conf.set('fs.azure.account.oauth.provider.type', 'org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider')
spark.conf.set('fs.azure.account.oauth2.client.id', client_id)
spark.conf.set('fs.azure.account.oauth2.client.secret', client_secret)
spark.conf.set('fs.azure.account.oauth2.client.endpoint', f'https://login.microsoftonline.com/{directory_id}/oauth2/token')

# list all items in container / directory
display(dbutils.fs.ls('abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/'))

Out[5]: [FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200919074931542.csv', name='2020-09-23_Report_20200919074931542.csv', size=17160),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200919074931542.xlsx', name='2020-09-23_Report_20200919074931542.xlsx', size=17494),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200920075127285.csv', name='2020-09-23_Report_20200920075127285.csv', size=17013),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200920075127285.xlsx', name='2020-09-23_Report_20200920075127285.xlsx', size=17342),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200921074101787.xlsx', name='2020-09-23_Report_20200921074101787.xlsx', size=17529),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200922073605265.xlsx', name='2020-09-23_Report_20200922073605265.xlsx', size=17532),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-23_Report_20200923080724434.xlsx', name='2020-09-23_Report_20200923080724434.xlsx', size=17554),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-24_Report_20200924073650323.xlsx', name='2020-09-24_Report_20200924073650323.xlsx', size=14942),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-25_Report_20200925073641577.xlsx', name='2020-09-25_Report_20200925073641577.xlsx', size=14921),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-26_Report_20200926074749154.xlsx', name='2020-09-26_Report_20200926074749154.xlsx', size=14920),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-27_Report_20200927075809253.xlsx', name='2020-09-27_Report_20200927075809253.xlsx', size=14883),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-28_Report_20200928073859699.xlsx', name='2020-09-28_Report_20200928073859699.xlsx', size=14905),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-29_Report_20200929084940041.xlsx', name='2020-09-29_Report_20200929084940041.xlsx', size=14911),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-09-30_Report_20200930090003428.xlsx', name='2020-09-30_Report_20200930090003428.xlsx', size=14921),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-01_Report_20201001091119922.xlsx', name='2020-10-01_Report_20201001091119922.xlsx', size=14913),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-02_Report_20201002090004954.xlsx', name='2020-10-02_Report_20201002090004954.xlsx', size=14906),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-03_Report_20201003074820539.xlsx', name='2020-10-03_Report_20201003074820539.xlsx', size=14917),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-04_Report_20201004074816138.xlsx', name='2020-10-04_Report_20201004074816138.xlsx', size=14843),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-05_Report_20201005074014769.xlsx', name='2020-10-05_Report_20201005074014769.xlsx', size=14910),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-06_Report_20201006073516322.xlsx', name='2020-10-06_Report_20201006073516322.xlsx', size=14917),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-07_Report_20201007073500683.xlsx', name='2020-10-07_Report_20201007073500683.xlsx', size=14920),
 FileInfo(path='abfss://ecovacs@storagejamesleslie.dfs.core.windows.net/BestBuy/2020-10-08_Report_2020100807361

---
# Using dbutils

In [9]:
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

## 1. Listing files / directories

In [11]:
# Mount the "transactions container"
dbutils.fs.mount(
    source = f'abfss://transactions@{account_name}.dfs.core.windows.net/',
    mount_point = f'/mnt/transactions',
    extra_configs = configs)

# list all mounted directories
display(dbutils.fs.ls('/mnt'))

Out[7]: [FileInfo(path='dbfs:/mnt/ecovacs/', name='ecovacs/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/', name='transactions/', size=0)]

In [12]:
# list all items in the 2020 directory
display(dbutils.fs.ls('/mnt/transactions/2020'))

Out[9]: [FileInfo(path='dbfs:/mnt/transactions/2020/p01/', name='p01/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p02/', name='p02/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p03/', name='p03/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p04/', name='p04/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p05/', name='p05/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p06/', name='p06/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p07/', name='p07/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p08/', name='p08/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p09/', name='p09/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p10/', name='p10/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p11/', name='p11/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p12/', name='p12/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p13/', name='p13/', size=0)]

## 2. Creating new directories

In [14]:
# create a new directory
dbutils.fs.mkdirs('/mnt/transactions/2020/p14')

# check that it appears in the mount point
display(dbutils.fs.ls('/mnt/transactions/2020'))

Out[11]: [FileInfo(path='dbfs:/mnt/transactions/2020/p01/', name='p01/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p02/', name='p02/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p03/', name='p03/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p04/', name='p04/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p05/', name='p05/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p06/', name='p06/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p07/', name='p07/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p08/', name='p08/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p09/', name='p09/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p10/', name='p10/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p11/', name='p11/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p12/', name='p12/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p13/', name='p13/', size=0),
 FileInfo(path='dbfs:/mnt/transactions/2020/p14/', name='p14/', size=0)]

In [15]:
# does it appear in the data lake too?
display(dbutils.fs.ls('abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/'))

Out[12]: [FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p01/', name='p01/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p02/', name='p02/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p03/', name='p03/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p04/', name='p04/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p05/', name='p05/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p06/', name='p06/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p07/', name='p07/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p08/', name='p08/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p09/', name='p09/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p10/', name='p10/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p11/', name='p11/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p12/', name='p12/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p13/', name='p13/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p14/', name='p14/', size=0)]

## 3. Remove directory

In [17]:
# remove the new directory and all its contents
dbutils.fs.rm('/mnt/transactions/2020/p14', recurse=True)

# is it gone?
display(dbutils.fs.ls('abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/'))

Out[13]: [FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p01/', name='p01/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p02/', name='p02/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p03/', name='p03/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p04/', name='p04/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p05/', name='p05/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p06/', name='p06/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p07/', name='p07/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p08/', name='p08/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p09/', name='p09/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p10/', name='p10/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p11/', name='p11/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p12/', name='p12/', size=0),
 FileInfo(path='abfss://transactions@storagejamesleslie.dfs.core.windows.net/2020/p13/', name='p13/', size=0)]